In [19]:
# coding:utf-8
# 关键词是自然语言处理，能翻页

import re
import urllib.request
import chardet
from bs4 import BeautifulSoup as bs
import requests


# 提取网页正文，放入txt中
def remove_js_css(content):
    """ remove the the javascript and the stylesheet and the comment content (<script>....</script> and <style>....</style> <!-- xxx -->) """
    content = content.decode('utf-8')
    r = re.compile(r'''<script.*?</script>''', re.I | re.M | re.S)
    s = r.sub('', content)
    r = re.compile(r'''<style.*?</style>''', re.I | re.M | re.S)
    s = r.sub('', s)
    r = re.compile(r'''<!--.*?-->''', re.I | re.M | re.S)
    s = r.sub('', s)
    r = re.compile(r'''<meta.*?>''', re.I | re.M | re.S)
    s = r.sub('', s)
    r = re.compile(r'''<ins.*?</ins>''', re.I | re.M | re.S)
    s = r.sub('', s)
    return s


def remove_empty_line(content):
    """remove multi space """
    r = re.compile(r'''^\s+$''', re.M | re.S)
    s = r.sub('', content)
    r = re.compile(r'''\n+''', re.M | re.S)
    s = r.sub('\n', s)
    return s


def remove_any_tag(s):
    s = re.sub(r'''<[^>]+>''', '', s)
    return s.strip()


def remove_any_tag_but_a(s):
    text = re.findall(r'''<a[^r][^>]*>(.*?)</a>''', s, re.I | re.S | re.S)
    text_b = remove_any_tag(s)
    return len(''.join(text)), len(text_b)


def remove_image(s, n=50):
    image = 'a' * n
    r = re.compile(r'''<img.*?>''', re.I | re.M | re.S)
    s = r.sub(image, s)
    return s


def remove_video(s, n=1000):
    video = 'a' * n
    r = re.compile(r'''<embed.*?>''', re.I | re.M | re.S)
    s = r.sub(video, s)
    return s


def sum_max(values):
    cur_max = values[0]
    glo_max = -999999
    left, right = 0, 0
    for index, value in enumerate(values):
        cur_max += value
        if (cur_max > glo_max):
            glo_max = cur_max
            right = index
        elif (cur_max < 0):
            cur_max = 0

    for i in range(right, -1, -1):
        glo_max -= values[i]
        if abs(glo_max < 0.00001):
            left = i
            break
    return left, right + 1


def method_1(content, k=1):
    if not content:
        return None, None, None, None
    tmp = content.split('\n')
    group_value = []
    for i in range(0, len(tmp), k):
        group = '\n'.join(tmp[i:i + k])
        group = remove_image(group)
        group = remove_video(group)
        text_a, text_b = remove_any_tag_but_a(group)
        temp = (text_b - text_a) - 8
        group_value.append(temp)
    left, right = sum_max(group_value)
    return left, right, len('\n'.join(tmp[:left])), len('\n'.join(tmp[:right]))


def extract(content):
    content = remove_empty_line(remove_js_css(content))
    left, right, x, y = method_1(content)
    return '\n'.join(content.split('\n')[left:right])


# 输入url，将其新闻页的正文输入txt
def extract_news_content(web_url, file_name):
    headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"
}
    request = requests.get(web_url, headers=headers)
    #opener = urllib.request.build_opener()
    #html = opener.open(request).read()
    html = request.content
    infoencode = chardet.detect(html)['encoding']  # 提取网页的编码
    if html != None and infoencode != None:  # 提取内容不为空，error.或者用else
        html = html.decode(infoencode, 'ignore')
        soup = bs(html,'lxml')
        content = soup.renderContents()
        content_text = extract(content)  # 提取新闻网页中的正文部分，化为无换行的一段文字
        content_text = re.sub(" ", " ", content_text)
        content_text = re.sub("&gt;", "", content_text)
        content_text = re.sub("&quot;", '""', content_text)
        content_text = re.sub("<[^>]+>", "", content_text)
        content_text = re.sub("\n", "", content_text)
        file = open(file_name, 'a')  # append
        file.write(content_text)
        file.close()
        print("输入成功！")


# 抓取百度新闻搜索结果:中文搜索，前10页，url：key=关键词
def search(key_word):
    headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"
}
    search_url = 'http://news.baidu.com/ns?word=key_word&tn=news&from=news&cl=2&rn=20&ct=1'
    search_url = search_url.replace('key_word',key_word)
    print(search_url)
    req = requests.get(search_url, headers=headers)
    #print(req.text)
    real_visited = 0
    http_url=None
    url = "http://www.baidu.com"
    visited_url_list = []
    for count in range(2):  # 前2页
        if count>0:
            req=requests.get(http_url)
        html = req.content
        soup = bs(html, 'lxml')
        content = soup.findAll( "div", {"class":"result-op"})  # resultset object
        #print(content)
        next=soup.findAll("a",{"class":"n"})
        #print(next)
        dddd=len(next) - 1
        if dddd<0:
            print("未搜索到")
            return
        ssss=next[dddd].get('href')
        http_url=url+ssss
        print(http_url)
        num = len(content)
        print(str(num))
        for i in range(num):
            p_str = content[i].find('a')  # if no result then nontype object
            contenttitle = p_str.renderContents()
            contenttitle = contenttitle.decode('utf-8', 'ignore')  # need it
            contenttitle = re.sub("<[^>]+>", "", contenttitle)
            contentlink = str(p_str.get("href"))
            print(contentlink)
            # 存放顺利抓取的url，对比
            visited_url = open('./visited.txt', 'r')  # 是否已经爬过
            visited_url_list = visited_url.readlines()
            visited_url.close()  # 及时close
            exist = 0
            if contentlink in visited_url_list:
                exist = 1
            else:  # 如果未被访问url
                real_visited += 1
                file_name = './newscn/%d.txt' % (real_visited)
                extract_news_content(contentlink, file_name)  # 写入文件
                visited_url_list.append(contentlink)
                visited_url = open('./visited.txt', 'a')  # 标记为已访问，永久存防止程序停止后丢失
                visited_url.write(contentlink + u'\n')
                visited_url.close()
            if len(visited_url_list) >= 120:
                print("解析下一页")
        if len(visited_url_list) >= 120:
            print("解析下一页")


if __name__ == '__main__':
    #key_word = input('input key word:')
    search("自然语言处理")



http://news.baidu.com/ns?word=自然语言处理&tn=news&from=news&cl=2&rn=20&ct=1
http://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&rsv_dl=ns_pc&word=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86&x_bfe_rqs=03E8000000000000000044&x_bfe_tjscore=0.100000&tngroupname=organic_news&newVideo=12&goods_entry_switch=1&pn=10
10
http://yuanchuang.10jqka.com.cn/20211223/c635373551.shtml
输入成功！
http://news.10jqka.com.cn/20211223/c635373551.shtml
输入成功！
https://www.163.com/dy/article/GRR7OG5J051480KF.html
输入成功！
http://dy.163.com/article/GRTCGB4505340BZM.html
输入成功！
http://stock.stockstar.com/RB2021122300007616.shtml
输入成功！
https://baijiahao.baidu.com/s?id=1719007876425418276&wfr=spider&for=pc
输入成功！
http://k.sina.com.cn/article_7628337882_1c6af32da001010ptr.html
输入成功！
https://baijiahao.baidu.com/s?id=1719178014796245420&wfr=spider&for=pc
输入成功！
http://baijiahao.baidu.com/s?id=1718935882272316070&wfr=spider&for=pc
输入成功！
http://baijiahao.baidu.com/s?id=1718996625200801734&wfr=spider&for=pc
输入成功！
http://www.baid

UnicodeEncodeError: 'gbk' codec can't encode character '\xa0' in position 272: illegal multibyte sequence

In [25]:
# coding:utf-8
# 文本摘要方法有很多，主要分为抽取式和生成式，应用比较多的是抽取式，也比较简单，就是从文本中抽取重要的句子或段落。本方法主要是利用句子中的关键词的距离，主要思想和参考来自阮一峰的网络日志http://www.ruanyifeng.com/blog/2013/03/automatic_summarization.html
import nltk
import numpy
import jieba
import codecs
# N=100#单词数量
# CLUSTER_THRESHOLD=5#单词间的距离
# TOP_SENTENCES=5#返回的top n句子

#分句
def sent_tokenizer(texts):
    start=0
    i=0#每个字符的位置
    sentences=[]
    punt_list='.!?。！？' #',.!?:;~，。！？：；～'.decode('utf8')
    token=""
    for text in texts:
        if text in punt_list and token not in punt_list: #检查标点符号下一个字符是否还是标点
            sentences.append(texts[start:i+1])#当前标点符号位置
            start=i+1#start标记到下一句的开头
            i+=1
        else:
            i+=1#若不是标点符号，则字符位置继续前移
            token=list(texts[start:i+2]).pop()#取下一个字符
    if start<len(texts):
        sentences.append(texts[start:])#这是为了处理文本末尾没有标点符号的情况
    return sentences

#摘要
def summarize(text):
    N = 100
    TOP_SENTENCES = 5
    sentences=sent_tokenizer(text)
    words=[w for sentence in sentences for w in jieba.cut(sentence) if len(w)>1 and w!='\t']
    wordfre=nltk.FreqDist(words)
    topn_words=[w[0] for w in sorted(wordfre.items(),key=lambda d:d[1],reverse=True)][:N]
    scored_sentences=_score_sentences(sentences,topn_words)
    #approach 1,利用均值和标准差过滤非重要句子
    avg=numpy.mean([s[1] for s in scored_sentences])#均值
    std=numpy.std([s[1] for s in scored_sentences])#标准差
    mean_scored=[(sent_idx,score) for (sent_idx,score) in scored_sentences if score>(avg+0.5*std)]
    #approach 2，返回top n句子
    top_n_scored=sorted(scored_sentences,key=lambda s:s[1])[-TOP_SENTENCES:]
    top_n_scored=sorted(top_n_scored,key=lambda s:s[0])
    top_n_summary = [sentences[idx] for (idx, score) in top_n_scored]
    mean_scored_summary = [sentences[idx] for (idx, score) in mean_scored]
    di={"top_n_summary":top_n_summary, "mean_scored_summary":mean_scored_summary}
    return di

 #句子得分
def _score_sentences(sentences,topn_words):
    CLUSTER_THRESHOLD = 5
    scores=[]
    sentence_idx=-1
    for s in [list(jieba.cut(s)) for s in sentences]:
        sentence_idx+=1
        word_idx=[]
        for w in topn_words:
            try:
                word_idx.append(s.index(w))#关键词出现在该句子中的索引位置
            except ValueError:#w不在句子中
                pass
        word_idx.sort()
        if len(word_idx)==0:
            continue
        #对于两个连续的单词，利用单词位置索引，通过距离阀值计算簇
        clusters=[]
        cluster=[word_idx[0]]
        i=1
        while i<len(word_idx):
            if word_idx[i]-word_idx[i-1]<CLUSTER_THRESHOLD:
                cluster.append(word_idx[i])
            else:
                clusters.append(cluster[:])
                cluster=[word_idx[i]]
            i+=1
        clusters.append(cluster)
        #对每个族打分，每个族类的最大分数是对句子的打分
        max_cluster_score=0
        for c in clusters:
            significant_words_in_cluster=len(c)
            total_words_in_cluster=c[-1]-c[0]+1
            score=1.0*significant_words_in_cluster*significant_words_in_cluster/total_words_in_cluster
            if score>max_cluster_score:
                max_cluster_score=score
        scores.append((sentence_idx,max_cluster_score))
    return scores;


if __name__=='__main__':
    text = []
    for i in range(3):
        filename = './newscn/%d.txt' % (i+1)
        file = text.append(filename)
        # print filename
    #
    print(text)
    for item in text:
        print(item)
        file_ = open(item).read()
        print(file_)
        dict = summarize(file_)
        file = open(item, 'a+')
        file.write("\n\n摘要：\n")
        print('-----------approach 1-------------')
        for sent in dict['top_n_summary']:
            file.write(sent)
            print(sent)
        file.close()


        #print('-----------approach 2-------------')
        #for sent in dict['mean_scored_summary']:
        #    print(sent)


['./newscn/1.txt', './newscn/2.txt', './newscn/3.txt']
./newscn/1.txt
鸥玛软件:公司与山东大学合作在自然语言处理、机器学习、智能识别...同花顺财经昨天17:03同花顺(300033)金融研究中心12月23日讯,有投资者向鸥玛软件(301185)提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。这些应也是元宇宙科技创新的基本...百度快照鸥玛软件:公司与山东大学合作在自然语言处理、机器学习、智能识别...同花顺财经昨天17:03同花顺金融研究中心12月23日讯,有投资者向鸥玛软件提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。这些应也是元宇宙科技创新的基本步骤?谢谢尊敬的董...百度快照从愿景到现实:人工智能在医疗保健领域的兴起 | 元宇宙网易新闻昨天12:17AI 的核心是机器学习,它由三个认知节点组成:计算机视觉、自然语言处理和数据推理。计算机视觉是人工智能的“眼睛”,因为它能够比人类更快地识别数字图像中的视觉模式、对象、场景和活动。自然语言处理是指识别和理解口语的技术。结构化数据...百度快照...在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等...证券之星昨天16:29公司与山东大学通过共建“山东大学-鸥玛软件人工智能创新研究院”, 在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合攻关研究、产业化应用、人才培养等方面进行深度合作。感谢您的关注与支持!百度快照科学家开发新机器学习模型,让计算机“读懂”人类对话网易前天16:15在研究和分析人与人对话的时候,研究人员发现许多自然语言处理算法,只有在文本具有清晰结构时才能正常运行,例如当一个人用完整的句子说话的时候。然而,在现实生活中,人们很少说话如此正式,这使得系统很难准确确定句子的开头和结尾。 百度快照百度首席技术官王海峰:深耕自然语言处理近30年,推进AI融合创新人民资讯12月13日该技术旨在突破跨模态语义理解瓶颈，让机器能够像人一样，通过语言、听觉、视觉等获得对真实世界的统一认知，实现对复杂场景的理解。作为此项技术第一完成人，王海峰在《对

-----------approach 1-------------
同花顺财经昨天17:03同花顺(300033)金融研究中心12月23日讯,有投资者向鸥玛软件(301185)提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。
同花顺财经昨天17:03同花顺金融研究中心12月23日讯,有投资者向鸥玛软件提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。
同花顺财经昨天17:03同花顺(300033)金融研究中心12月23日讯,有投资者向鸥玛软件(301185)提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。
同花顺财经昨天17:03同花顺金融研究中心12月23日讯,有投资者向鸥玛软件提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。
百度快照【金猿技术展】专利智能语义检索——基于预训练的语义排序新浪新闻前天12:27通过深度学习、自然语言处理以及预训练语言模型等前沿人工智能技术的运用,实现在海量全球多语言专利文本中进行自动化、智能化的数据分析与文本挖掘,进一步实现深层次语义分析,为用户提供更加精准地语义检索服务。
./newscn/3.txt
鸥玛软件:公司与山东大学合作在自然语言处理、机器学习、智能识别...同花顺财经昨天17:03同花顺(300033)金融研究中心12月23日讯,有投资者向鸥玛软件(301185)提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。这些应也是元宇宙科技创新的基本...百度快照鸥玛软件:公司与山东大学合作在自然语言处理、机器学习、智能识别...同花顺财经昨天17:03同花顺金融研究中心12月23日讯,有投资者向鸥玛软件提问, 从新闻公告中得知公司在自然语言处理、机器学习、智能识别、人机交互及虚拟现实等人工智能技术方面联合山东大学攻关研究、深度合作。这些应也是元宇宙科技创新的基本步骤?谢谢尊敬的董...百度快照从愿景到现实:人工智

In [27]:
import re
def getvalue(direct):
    return direct["freq"]
a = []
for i in range(1,4):
    filename = './newscn/%d.txt' % (i)
    file = a.append(filename)
    print(filename)

# print str(a)
b = []
lists=[]
for n in a:
    print(n)
    file_ = open(n).read()
    # print file_
    freq = len(re.findall("自然语言处理",file_))
    print(freq) # 单个文档的关键词词频
    # file.close()
    dir={"file_dir":n,"freq":freq}
    lists.append(dir)
    freq_ = b.append(freq)
#print lists[0]
lists.sort(key=getvalue,reverse=True)
print(lists)


./newscn/1.txt
./newscn/2.txt
./newscn/3.txt
./newscn/1.txt
112
./newscn/2.txt
43
./newscn/3.txt
44
[{'file_dir': './newscn/1.txt', 'freq': 112}, {'file_dir': './newscn/3.txt', 'freq': 44}, {'file_dir': './newscn/2.txt', 'freq': 43}]
